In [ ]:
# pip install -U flask-cors tensorflow nltk
# nltk.download('punkt')
# nltk.download('wordnet')

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import random

lemmatizer = WordNetLemmatizer()

     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
      --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.5 MB 1.2 MB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.5 MB 1.1 MB/s eta 0:00:02
     --------- ------------------------------ 0.3/1.5 MB 2.0 MB/s eta 0:00:01
     --------------- ------------------------ 0.6/1.5 MB 2.5 MB/s eta 0:00:01
     ----------------- ---------------------- 0.7/1.5 MB 2.7 MB/s eta 0:00:01
     ------------------------ --------------- 0.9/1.5 MB 3.0 MB/s eta 0:00:01
     ---------------------------- ----------- 1.1/1.5 MB 2.8 MB/s eta 0:00:01
     ------------------------------------ --- 1.4/1.5 MB 3.3 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 3.4 MB/s eta 0:00:00
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be

In [5]:
from tensorflow.keras.models import load_model
import json

model = load_model('chatbot_model.h5')
data_file = open('intents.json').read()
intents = json.loads(data_file)
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    print("sentence words: ", sentence_words)
    return sentence_words

In [8]:
def bow(sentence, words):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return(np.array(bag))

In [9]:
def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    
    result = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    result.sort(key=lambda x: x[1], reverse= True)
    result_list = []
    for r in result:
        result_list.append({"intent": classes[r[0]], "probability": str(r[1])})
        
    return result_list

In [10]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
    return result

In [11]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    try:
        res = getResponse(ints, intents)
    except:
        res = "I am really sorry, I don't have the knowlegde to answer this."
    return res
    

In [17]:
from flask import Flask, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/", methods = ['GET'])
def hello():
    return jsonify({"Server": "Sucessfully"})

def decrypt(msg):
    string = msg
    new_string = string.replace("+", " ")
    return new_string

@app.route("/query/<sentence>")
def query_chatbot(sentence):
    dec_msg = decrypt(sentence)
    
    response = chatbot_response(dec_msg)
    
    json_obj = jsonify({"top":{"res": response}})
    
    return json_obj

In [10]:
# import logging
# from flask import Flask, jsonify
# from flask_ngrok import run_with_ngrok
# from flask_cors import CORS

# app = Flask(__name__)
# run_with_ngrok(app)
# CORS(app)

# @app.route("/", methods = ['GET'])
# def index():
#     return jsonify({"Server": "Sucessfully"})

# def decrypt(msg):
#     string = msg
#     new_string = string.replace("+", " ")
#     return new_string

# @app.route("/query/<sentence>")
# def query_chatbot(sentence):
#     dec_msg = decrypt(sentence)
    
#     response = chatbot_response(dec_msg)
    
#     json_obj = jsonify({"top":{"res": response}})
    
#     return json_obj

# if __name__ == '__main__':
#     app.run()

In [22]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Aug/2023 21:20:34] "GET / HTTP/1.1" 200 -


sentence words:  ['how', 'are', 'you']
1/1 [==============================] - 0s 79ms/step


127.0.0.1 - - [05/Aug/2023 21:20:45] "GET /query/How+are+you HTTP/1.1" 200 -


sentence words:  ['hello']
1/1 [==============================] - 0s 16ms/step


...

127.0.0.1 - - [05/Aug/2023 21:22:00] "GET /query/Can+you+tell+me+about+the+IUB+club HTTP/1.1" 200 -


sentence words:  ['thank', 'you', '.']
1/1 [==============================] - 0s 15ms/step


127.0.0.1 - - [05/Aug/2023 21:22:14] "GET /query/Thank+you. HTTP/1.1" 200 -


sentence words:  ['bye']
1/1 [==============================] - 0s 16ms/step


127.0.0.1 - - [05/Aug/2023 21:22:18] "GET /query/bye HTTP/1.1" 200 -
